In [1]:
import csv
import re
import json
from tqdm import tqdm

## Корпус
#### его содержимое, формат хранения + тегсет

Корпус хранится в csv-файле, поскольку так было удобнее размечать данные. Ниже функция, переводящая в формат списка "слово+часть речи", более удобный для сравнения

Также есть json-файл с делением на предложения, а не на токены, чтобы прогонять через модели, учитывающие контекст

В корпус входят различные предложения, в основном, из новостей. Представлены различные части речи, омонимия, разрешаемая контекстом, сокращения, аббревиатуры, слова, отсутствующие в словарях

In [2]:
def csv_to_list(tokens):
    with open('corpus.csv', 'r', encoding='utf-8') as csvfile:
        datareader = csv.reader(csvfile)
        next(datareader)
        for row in datareader:
            tokens.append((row[0], row[1]))
        return

In [3]:
tokens = []
csv_to_list(tokens)

In [4]:
with open('corpus.json', encoding="utf-8") as file:
    corpus = json.load(file)

In [5]:
print(len(tokens), '- количество токенов')

201 - количество токенов


In [6]:
our_tags = ['noun','adj','verb','adv','pron','comp','num','prep','conj','prcl','intj','apro']

Почему именно такой набор частей речи?

- причастия и деепричастия включены в глагол, так как если теггер помечает причастие как глагол, то нет смысла проверять, понимает ли он, что за форма глагола, ведь причастия имеют окончания, очень отличные от обычных глагольных финитных. Гораздо важнее проверить, что теггер отличает причастие от прилагательного, что данное обобщение не мешает делать
- краткая и полная форма прилагательных в одном теге: по аналогичным причинам, хочется проверить скорее различение кратких прилагательных и существительных
- местоимения pron и apro (местоимения-существительные и местоимения-прилагательные): с остальными классами местоимений обозначения у разных моделей очень разнятся. При этом так как местоимения - это закрытый класс со множеством супплетивных форм, их разбор легко сделать даже просто словарём, поэтому, вероятно, качественные теггеры уже сделали такой разбор, который считают правильным, и проверять это нет особого смысла
- порядковые числительные помечаются как прилагательные, поскольку Natasha никак не различает их
- компаратив объединяет сравнительные формы прилагательных и наречий, так как разбор таких случаев бывает спорным даже для человеческой разметки, а также Pymorphy не разделяет их

## Pymorphy

In [7]:
from pymorphy2 import MorphAnalyzer
pm = MorphAnalyzer()

Функция, приводящая тег к нужному формату:

In [8]:
pm_dict = {'ADJF': 'adj',
           'ADJS': 'adj',
           'INFN': 'verb',
           'PRTF': 'verb',
           'PRTS': 'verb',
           'GRND': 'verb',
           'NUMR': 'num',
           'ADVB': 'adv',
           'NPRO': 'pron',
          }

def pm_tags(gr):
    if 'Apro' in str(gr):
        return 'apro'
    tag = gr.POS
    if tag == None:
        return 'unkn'
    if tag.lower() in our_tags:
        return tag.lower()
    elif tag in pm_dict:
        new_tag = pm_dict[tag]        
        return new_tag
    return tag

Контекст не учитывается, поэтому смотрим по одному слову (и сразу сравниваем):

In [9]:
pm_c = 0
for token in tokens:
    a = pm.parse(token[0])
    word_tags = []
    for analyze in a:
        w_tag = pm_tags(analyze.tag)
        word_tags.append(str(w_tag))
    if token[1] in word_tags:
        pm_c += 1
    else:
        print('ошибка, оригинал:', token, ', варианты:', word_tags)

print('\n\n', pm_c/len(tokens),'- accuracy Pymorphy')

ошибка, оригинал: ('НИУ', 'noun') , варианты: ['unkn']
ошибка, оригинал: ('ВШЭ', 'noun') , варианты: ['unkn']
ошибка, оригинал: ('один', 'num') , варианты: ['apro', 'apro']


 0.9850746268656716 - accuracy Pymorphy


## Mystem

In [10]:
import os
from pymystem3 import Mystem
os.environ["MYSTEM_BIN"] = "C:\\mystem.exe"
ms = Mystem()

In [11]:
ms_dict = {'A': 'adj',
           'ADVPRO': 'adv',
           'ANUM': 'adj',
           'PART': 'prcl',
           'PR': 'prep',
           'S': 'noun',
           'SPRO': 'pron',
           'V': 'verb'
          }


def ms_tags(gr):
    if 'срав' in gr:
        return 'comp'
    
    tag = re.search(r'\w+', gr).group(0)
    
    if tag.lower() in our_tags:
        return tag.lower()
    
    elif tag in ms_dict:
        new_tag = ms_dict[tag]
        return new_tag
    
    return tag

In [12]:
ms_tokens = []
for sent in corpus:
    ms_an = ms.analyze(sent[0])
    sent_tokens = []
    for token in tqdm(ms_an):
        if 'analysis' in token:
            word = token['text']
            if len(token['analysis']) == 0:
                sent_tokens.append((word, 'unkn'))
            else:
                features = token['analysis'][0]['gr']
                tag = ms_tags(features)
                sent_tokens.append((word, tag))
    if len(sent_tokens) != sent[1]:
        #проверяем, нет ли предложений, в которых токенизация прошла не так, как вручную (по количеству токенов в предложении)
        print(sent, sent_tokens)
    else:
        ms_tokens.extend(sent_tokens)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<?, ?it/s]


In [13]:
print(len(ms_tokens), '- количество токенов') #проверка общего количества

201 - количество токенов


Теперь сравним с эталоном:

In [14]:
ms_c = 0
for i in range(len(tokens)):
    if tokens[i][1] == ms_tokens[i][1]:
        ms_c += 1
    else:
        print('ошибка:',tokens[i], ms_tokens[i])

ошибка: ('проверенным', 'verb') ('проверенным', 'adj')
ошибка: ('ВШЭ', 'noun') ('ВШЭ', 'unkn')
ошибка: ('ели', 'noun') ('ели', 'verb')
ошибка: ('несколько', 'num') ('несколько', 'adv')
ошибка: ('один', 'num') ('один', 'apro')
ошибка: ('все', 'prcl') ('все', 'pron')
ошибка: ('о', 'intj') ('о', 'prep')
ошибка: ('старче', 'noun') ('старче', 'comp')
ошибка: ('ли', 'prcl') ('ли', 'conj')
ошибка: ('зорко', 'adj') ('зорко', 'adv')


In [15]:
print(ms_c/len(tokens), '- accuracy Mystem')

0.9502487562189055 - accuracy Mystem


## Natasha

In [16]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [17]:
text = ' '.join(sent for (sent,num) in corpus)
doc = Doc(text)

In [18]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [19]:
nat_dict = {'ADP': 'prep',
            'PROPN': 'noun',
            'AUX': 'verb',
            'DET': 'apro',
            'PART': 'prcl',
            'SCONJ': 'conj',
            'CCONJ': 'conj',
            'PUNCT': 'punct'
           }


def nat_tags(token):
    if 'Degree' in token.feats and token.feats['Degree'] == 'Cmp':
        return 'comp'
    
    tag = token.pos
    if tag.lower() in our_tags:
        return tag.lower()
    
    elif tag in nat_dict:
        new_tag = nat_dict[tag]        
        return new_tag
    
    else:
        print(tag)
        return tag

In [20]:
nat_tokens = []
for sent in doc.sents:
    a = sent.morph
    for token in tqdm(a.tokens):
        tag = nat_tags(token)
        if tag not in ['punct']:
            nat_tokens.append((token.text, tag))

100%|███████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 14947.63it/s]


In [21]:
print(len(nat_tokens), ' - количество токенов') # проверка совпадения токенизации

201  - количество токенов


In [22]:
nat_c = 0
for i in range(len(tokens)):
    if tokens[i][1] == nat_tokens[i][1]:
        nat_c += 1
    else:
        print('ошибка:', tokens[i], nat_tokens[i])

ошибка: ('что', 'conj') ('что', 'pron')
ошибка: ('и', 'conj') ('и', 'prcl')
ошибка: ('научусь', 'verb') ('научусь', 'noun')
ошибка: ('тридцатое', 'adj') ('тридцатое', 'noun')
ошибка: ('Качавшая', 'verb') ('Качавшая', 'adj')
ошибка: ('его', 'apro') ('его', 'pron')
ошибка: ('Ответь', 'verb') ('Ответь', 'noun')
ошибка: ('о', 'intj') ('о', 'prep')
ошибка: ('светла', 'adj') ('светла', 'noun')
ошибка: ('зорко', 'adj') ('зорко', 'adv')


In [24]:
print(nat_c/len(tokens), '- accuracy Natasha')
print(ms_c/len(tokens), '- accuracy Mystem')
print(pm_c/len(tokens), '- accuracy Pymorphy')

0.9502487562189055 - accuracy Natasha
0.9502487562189055 - accuracy Mystem
0.9850746268656716 - accuracy Pymorphy


Результат: Pymorphy обогнал остальных, но только засчёт того, что предлагал несколько разборов. Так как для внедрения pos-теггера в предыдущую программу, нам нужно точно определять часть речи, Pymorphy будет работать значительно хуже, возьмём Natasha.

(продолжение в тетрадке nlp_1_new)